In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import torch
import gc
import Stemmer
import re
from tqdm import tqdm
import csv
import gc
from utils.graph import KGraphPreproc
from utils.graph.chain import GraphChain
from utils.llm.mistral import MistralLLM
from utils.prompt import GRAPH_QA_PROMPT, ENTITY_PROMPT
from utils.file import export_results_to_file
import os

In [32]:
clear_cache()

In [9]:
def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [8]:
l = 10

In [ ]:
for c, (i, r) in enumerate(tqdm(list(metaqa.iterrows()))):
    print(c, l)
    if c < l:
        continue
    print(i, c)
    break

  1%|          | 10/1000 [00:00<00:00, 54050.31it/s]

0 10
1 10
2 10
3 10
4 10
5 10
6 10
7 10
8 10
9 10
10 10
4764 10


: 

: 

In [2]:
metaqa = pd.read_csv(f"/datasets/MetaQA/3hop/test_1000.txt", header=None, index_col=0)
metaqa.rename(columns={1: "Question", 2: "Answers"}, inplace=True)
metaqa

,Question,Answers
0,,
9043,what are the genres of the films whose directo...,Drama|Comedy
2984,when did the movies starred by [First Daughter...,1986|1987|1984|1985|1982|1983|1979|1988|1989|2...
10223,what are the genres of the films whose actors ...,Drama|Action|Comedy|Adventure|Crime
10876,what are the genres of the movies whose writer...,Drama|Crime
10672,what are the languages spoken in the films who...,English|Italian
...,...,...
5563,what were the release years of the films that ...,2014|2001|2011|2010|2007
11098,the films that share writers with the film [Th...,Drama|Comedy|Thriller
7199,the movies that share writers with the movie [...,German


In [7]:
fbqa = pd.read_csv("/datasets/FreebaseQA/FbQA-eval-1000.csv")
def get_fbqa_data(question_row):
    """
    Takes in a dataset row and returns Q and A as strings
    """
    question = question_row.Questions.get("RawQuestion", None)
    parse = question_row.Questions.get("Parses", [None])[0]
    if not parse:
        print(f"error in question: {question}")
        return question, None
    answer = parse.get("Answers")
    return question, answer


In [ ]:

####### load the graph
fbkb_graph = KGraphPreproc.get_fbkb_graph()

In [3]:
from utils.prompt import GRAPH_QA_PROMPT, ENTITY_PROMPT

mistral = MistralLLM()
chain = GraphChain.from_llm(
    llm=mistral,
    graph=fbkb_graph,
    qa_prompt=GRAPH_QA_PROMPT,
    entity_prompt=ENTITY_PROMPT,
    verbose=False,
)

NameError: name 'fbkb_graph' is not defined

In [5]:
def get_response(prompt):
    global chain
    # del mistral
    gc.collect()
    torch.cuda.empty_cache()
    r = chain.invoke(prompt)
    return r["result"]

In [60]:
chain.exploration_depth = 1

In [8]:

###### tests

experiment_name = "kb2"
bline_path = f"/datasets/FreebaseQA/results/{experiment_name}.csv"
bline = pd.read_csv(bline_path)
l = len(bline)
baseline_results = list(bline.Model.values)

fbqa = pd.read_json("/datasets/FreebaseQA/FreebaseQA-eval.json")
# fbqa.Questions[0].get("RawQuestions", None)
results = []
for i, r in tqdm(list(fbqa.iterrows())):
    if i == 10:
        break
    if i < l:
        continue
    q, a = get_fbqa_data(r)
    # 
    # print("Question:", q)
    # print("Answer:", a)
    # print("Prompt:", prompt)
    print(q)
    response = get_response(q)
    print("Model:", response)
    results.append(response)

    if i % 250 == 0:
        with open(bline_path, "w") as f:
            writer = csv.writer(f)
            writer.writerow(["Model"])
            for r in results:
                writer.writerow([str(r)])

with open(bline_path, "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Model"])
    for r in results:
        writer.writerow([str(r)])


  0%|          | 10/3996 [00:00<00:01, 2158.67it/s]


In [71]:
import bm25s
import Stemmer  # optional: for stemming
from utils.preprocessing import preprocess_text

ts = bm25s.tokenize(
    map(
        preprocess_text,
        triplets
    )
)
retriever = bm25s.BM25()
retriever.index(ts)

query = "Who produced the film 12 Angry Men, which was scripted by Reginald Rose, starred Henry Fonda and was directed by Sidney Lumet?"
print(preprocess_text(query))
query_tokens = bm25s.tokenize(preprocess_text(query), stemmer=stemmer)

results, scores = retriever.retrieve(query_tokens, k=len(ts))

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.3f}): {triplets[doc]}")

Split strings: 0it [00:00, ?it/s]

BM25S Count Tokens:   0%|          | 0/218 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/218 [00:00<?, ?it/s]

who produc the film 12 angri men which was script by reginald rose star henri fonda and was direct by sidney lumet


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 4.336): Sidney_Lumet /people/person/profession Film_Producer-GB
Rank 2 (score: 3.653): Sidney_Lumet /people/person/profession Television_producer-GB
